# Just to hide things

In [7]:
import pandas as pd
import numpy as np

# Further cleaning

In [ ]:
allowed_values = ['Amber', 'Green', 'Red', 'Amber/red', 'Amber/Green']
result['colour_rating_new'] = result['colour_rating_new'].apply(lambda x: x if x in allowed_values else np.nan)

In [ ]:
result = result.rename(columns={'colour_rating_new': 'colour_rating'})

In [ ]:
result['yearly_forecast_new'] = result['yearly_forecast'].combine_first(result['forecast_incl_NGC'])

In [ ]:
result = result.drop(columns=['yearly_forecast', 'forecast_incl_NGC'])

In [ ]:
result = result.rename(columns={'yearly_forecast_new': 'yearly_forecast'})

In [ ]:
result = result.drop(columns=['yearly_budget'])

In [ ]:
result = result.rename(columns={'total_baseline': 'yearly_budget'})
result = result.rename(columns={'TOTAL Baseline Benefits (Łm)': 'total_benefits'})

In [ ]:
column_order = ['project_name','department','colour_rating','description_aims','rating_comment','start_date','end_date','schedule_comment','yearly_budget','yearly_forecast','wlc_baseline_incl_NCG','variance_comment','budget_comment','year','report_category','project_number','total_benefits','benefits_comment']

In [ ]:
result = result[column_order]

In [ ]:
df_macro = pd.read_csv('../data/macro/GDP.csv',encoding='iso-8859-2', delimiter=';')

In [ ]:
df_macro

In [ ]:
df_macro.info()

In [ ]:
# Schritt 1: Identifizieren der relevanten Zeilen
remained_scheduled_df = df_franzi[df_franzi['schedule_comment'].str.contains('remained scheduled', case=False)]

# Schritt 2: Aktuellsten 'end_date'-Wert für jedes 'project_name' ermitteln
latest_end_dates = df_franzi.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'end_date'])

# Schritt 3: Zuweisen des aktuellsten 'end_date' an die relevanten Zeilen
for index, row in remained_scheduled_df.iterrows():
    latest_end_date = latest_end_dates[row['project_name']]
    if pd.isna(row['end_date']) and pd.notna(latest_end_date):
        df_franzi.at[index, 'end_date'] = latest_end_date

# Ergebnis anzeigen
df_franzi


In [ ]:
# Schritt 2: Aktuellsten 'end_date'-Wert für jedes 'project_name' ermitteln
latest_end_dates = df_franzi.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'start_date'])

# Schritt 3: Zuweisen des aktuellsten 'end_date' an die relevanten Zeilen
for index, row in remained_scheduled_df.iterrows():
    latest_end_date = latest_end_dates[row['project_name']]
    if pd.isna(row['start_date']) and pd.notna(latest_end_date):
        df_franzi.at[index, 'start_date'] = latest_end_date

# Ergebnis anzeigen
df_franzi

In [ ]:
    month_mapping = {
        'jan': '01', 'january': '01',
        'feb': '02', 'february': '02',
        'mar': '03', 'march': '03',
        'apr': '04', 'april': '04',
        'may': '05', 'may': '05',
        'jun': '06', 'june': '06',
        'jul': '07', 'july': '07',
        'aug': '08', 'august': '08',
        'sep': '09', 'september': '09',
        'oct': '10', 'october': '10',
        'nov': '11', 'november': '11',
        'dec': '12', 'december': '12'
    }



In [ ]:
def extract_date_after_to_or_on(comment):
    REGEX = "(to|is|on)\s*(\d{1,2})\s+(\w+)\s+(\d{2,4})"
    x = re.compile(REGEX)
    
    match_REGEX = x.search(comment)      
    
    if match_REGEX:
        day = match_REGEX.group(2)
        month = match_REGEX.group(3)
        year = match_REGEX.group(4)
        if len(day) == 1:
            day = '0'+ day
        month = month_mapping.get(month.lower())
        if len(year) == 2:
            year = '20' + year
        return year +'-'+ month +'-'+ day
    
for index, row in df_franzi.iterrows():
    if pd.isna(row['end_date']):
        extracted_date = extract_date_after_to_or_on(row['schedule_comment'])
        if extracted_date:
            df_franzi.at[index, 'end_date'] = extracted_date

df_franzi



In [ ]:
# Der Code zum Filtern von Zeilen, in denen 'start_date' oder 'end_date' 'NaN' ist
# und die 'schedule_comment' nicht mit 'Exempt' oder '\r\nExempt' beginnen

nan_dates_df = df_franzi[
    ((df_franzi['end_date'].isna())) & 
    (~df_franzi['schedule_comment'].str.match(r'^(Exempt|\r\nExempt)'))
][['start_date', 'end_date', 'schedule_comment']]

# Ausgabe des gefilterten DataFrames
nan_dates_df

In [ ]:
# Der Code zum Filtern von Zeilen, in denen 'start_date' oder 'end_date' 'NaN' ist
# und die 'schedule_comment' nicht mit 'Exempt' oder '\r\nExempt' beginnen

nan_dates_df_start = df_franzi[
    ((df_franzi['start_date'].isna())) & 
    (~df_franzi['schedule_comment'].str.match(r'^(Exempt|\r\nExempt)'))
][['start_date', 'schedule_comment']]

# Ausgabe des gefilterten DataFrames
nan_dates_df_start

In [ ]:
df1 = pd.read_pickle("../data/pickle/cleaned_2021_2023.pkl")
df2 = pd.read_pickle('../data/pickle/cleaned_2018_2020.pkl')
df3 = pd.read_pickle('../data/pickle/cleaned_2014-2017.pkl')
df4 = pd.read_csv('../data/macro/GDP.csv',encoding='iso-8859-2', delimiter=';')

In [ ]:
df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

In [ ]:
df = df.merge(df4, on='year', how='left')

In [ ]:
df.drop(columns='Unnamed: 0')

In [ ]:
df["colour_rating"] = df["colour_rating"].str.lower()

# Drop rows where 'project_name' contains 'Unnamed:' followed by other characters
df = df[~df['project_name'].str.contains(r'Unnamed:.*', na=False)]

# Optionally, reset the index of the cleaned DataFrame
df = df.reset_index(drop=True)

## Imputing

In [ ]:
latest_start_dates = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'start_date'])

In [ ]:
for index, row in df.iterrows():
    latest_start_date = latest_start_dates[row['project_name']]
    if pd.isna(row['start_date']) and pd.notna(latest_start_date):
        df.at[index, 'start_date'] = latest_start_date

In [ ]:
latest_benefits = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'total_benefits'])

for index, row in df.iterrows():
    latest_benefit = latest_benefits[row['project_name']]
    if pd.isna(row['total_benefits']) and pd.notna(latest_benefit):
        df.at[index, 'total_benefits'] = latest_benefit

In [ ]:
latest_categories = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'report_category'])

for index, row in df.iterrows():
    latest_category = latest_categories[row['project_name']]
    if pd.isna(row['report_category']) and pd.notna(latest_category):
        df.at[index, 'report_category'] = latest_category

In [ ]:
latest_numbers = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'project_number'])

for index, row in df.iterrows():
    latest_number = latest_numbers[row['project_name']]
    if pd.isna(row['project_number']) and pd.notna(latest_number):
        df.at[index, 'project_number'] = latest_number

In [ ]:
df['description_aims'].unique().tolist()

In [ ]:
df_with_dummies = pd.get_dummies(df, columns=['department', 'colour_rating'], drop_first=True)


In [ ]:
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])
# Calculate the duration in years
df["year_duration"] = (df["end_date"] - df["start_date"]).dt.days / 365.25

# Optionally, round the duration to a specific number of decimal places
df["year_duration"] = df["year_duration"].round(2)

In [ ]:
df['start_year'] = df['start_date'].dt.year
df['end_year'] = df['end_date'].dt.year

## Add macro data

In [ ]:
macro_columns = pd.read_csv('../data/macro/GDP.csv', encoding='iso-8859-2', delimiter=';')

In [ ]:
df = df.drop(columns=['GDP', 'annual_earning_ft', 'unemployment_rate', 'youth_unemployment_rate', 'inflation_rate', 'population', 'gov_debt', 'tax_revenue', 'revenue_excl_grants', 'grants_and_other_revenue'])

In [ ]:
df = df.merge(macro_columns, on='year', how='left')

## Final Pickle

In [8]:
df = pd.read_pickle('../data/pickle/final_pickle.pkl')

## Preprocessing

In [ ]:
df = df.drop(columns=['description_aims', 'rating_comment', 'start_date',
       'end_date', 'schedule_comment', 'yearly_budget', 'yearly_forecast', 'variance_comment', 'budget_comment',
       'report_category', 'project_number', 'total_benefits',
       'benefits_comment', 'population', 'revenue_excl_grants', 'grants_and_other_revenue'])

In [ ]:
df['year'].unique()

In [ ]:
df['year'] = df['year'].astype(int)

In [ ]:
df = df.dropna(subset=['year'])

In [ ]:
# colour_rating_columns = [col for col in df.columns if col.startswith('colour_rating_')]
# department_columns = [col for col in df.columns if col.startswith('department_')]

# def latest_value(df, column):
#     try:
#         return df.loc[df['year'].idxmax(), column]
#     except Exception as e:
#         print(f"Error in Group {df.name} for column {column}: {e}")
#         return None

agg_dict = {
    'wlc_baseline_incl_NCG': 'max',
    'year_duration': 'max',
    'end_year': 'max',
    'start_year': 'min',
    'year': 'max',
    'GDP': 'mean',
    'annual_earning_ft': 'mean',
    'unemployment_rate': 'mean',
    'youth_unemployment_rate': 'mean',
    'inflation_rate': 'mean',
    'gov_debt': 'mean',
    'tax_revenue': 'mean',
    'department_CO': lambda x: x.mode()[0],
    'department_CPS': lambda x: x.mode()[0],
    'department_DBT': lambda x: x.mode()[0],
    'department_DCMS': lambda x: x.mode()[0],
    'department_DEFRA': lambda x: x.mode()[0],
    'department_DEFRA & DFT': lambda x: x.mode()[0],
    'department_DESNZ': lambda x: x.mode()[0],
    'department_DFE': lambda x: x.mode()[0],
    'department_DFID': lambda x: x.mode()[0],
    'department_DFT': lambda x: x.mode()[0],
    'department_DHSC': lambda x: x.mode()[0],
    'department_DLUHC': lambda x: x.mode()[0],
    'department_DSIT': lambda x: x.mode()[0],
    'department_DWP': lambda x: x.mode()[0],
    'department_FCDO': lambda x: x.mode()[0],
    'department_HMLR': lambda x: x.mode()[0],
    'department_HMRC': lambda x: x.mode()[0],
    'department_HMT': lambda x: x.mode()[0],
    'department_HO': lambda x: x.mode()[0],
    'department_MOD': lambda x: x.mode()[0],
    'department_MOJ': lambda x: x.mode()[0],
    'department_NCA': lambda x: x.mode()[0],
    'department_NS&I': lambda x: x.mode()[0],
    'department_ONS': lambda x: x.mode()[0],
    'department_VOA': lambda x: x.mode()[0],
    'colour_rating_amber/green': lambda x: x.mode()[0],
    'colour_rating_amber/red': lambda x: x.mode()[0],
    'colour_rating_green': lambda x: x.mode()[0],
    'colour_rating_red': lambda x: x.mode()[0],
    'colour_rating_reset': lambda x: x.mode()[0]
}

# for col in colour_rating_columns:
#     agg_dict[col] = lambda x, col=col: latest_value(x, col)

# for col in department_columns:
#     agg_dict[col] = lambda x, col=col: latest_value(x, col)

df = df.groupby('project_name').agg(agg_dict)


In [ ]:
df = df.drop(columns=['revenue_excl_grants','grants_and_other_revenue'])

In [ ]:
df = df.drop(columns='year')

In [ ]:
df = df.drop(columns=['year_duration', 'end_year','year', 'youth_unemployment_rate'])

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = df.drop(columns=['wlc_baseline_incl_NCG'])
y = df['wlc_baseline_incl_NCG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print(coefficients)


In [ ]:
from xgboost import XGBRegressor

X = df.drop(columns=['wlc_baseline_incl_NCG'])
y = df['wlc_baseline_incl_NCG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_reg = XGBRegressor(objective='reg:squarederror', random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f'Best Params: {best_params}')
print(f'Best negative MSE (Training): {best_score}')

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (Test): {mse}')


In [ ]:
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

In [ ]:
from bayes_opt import BayesianOptimization
from hyperopt import hp, fmin, tpe

In [ ]:
from sklearn.datasets import make_regression

X = df.drop(columns=['wlc_baseline_incl_NCG'])
y = df['wlc_baseline_incl_NCG']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function to minimize
def xgb_eval(max_depth, learning_rate, subsample, colsample_bytree):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'n_estimators': 100,
        'objective': 'reg:squarederror',
        'random_state': 42,
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

# Define the search space
pbounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
}

# Perform Bayesian optimization
optimizer = BayesianOptimization(f=xgb_eval, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=25)

# Print the best hyperparameters and score
print(f"Best hyperparameters: {optimizer.max['params']}")
print(f"Best score: {optimizer.max['target']:.4f}")